# Simple Multlayer Perceptron

## Load Data

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
def load_data(location, target_last = True, target = None):
    data = pd.read_csv('data/pima-indians-diabetes.csv', header = -1)
    t = data.shape[1] - 1
    X = data.drop(t,axis = 1).as_matrix()
    Y = data[t].as_matrix()
    return X, Y, data

In [4]:
X,Y, data = load_data(location = 'data/pima-indians-diabetes.csv')

In [5]:
data.shape

(768, 9)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=11)

## Keras

In [17]:
from keras.models import Sequential
from keras.layers import Dense

In [60]:
model = Sequential()
model.add(Dense(12, input_dim = 8 , activation= 'relu'))
model.add(Dense(100,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=10,verbose= 0)

In [62]:
pred = model.predict_classes(X_test)

 32/254 [==>...........................] - ETA: 0s

In [63]:
print(classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.79      0.91      0.84       165
          1       0.76      0.54      0.63        89

avg / total       0.78      0.78      0.77       254



## H20

In [80]:
import h2o as h
from h2o.automl import H2OAutoML

In [76]:
h.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,04 secs
H2O cluster version:,3.15.0.4068
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_mrd_3apsmf
H2O cluster total nodes:,1
H2O cluster free memory:,3.544 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [90]:
data = data.rename_axis({0:'0',1:'1',2:'2',3:"3",4:"4",5:'5',6:'6',7:'7',8:'target'},axis=1)

In [92]:
hf = h.H2OFrame(data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [169]:
train, valid, test = hf.split_frame([.7,.15],seed = 1234)
X = hf.col_names
X.remove('target')
y = 'target'

In [170]:
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
aml = H2OAutoML()
aml.train(x = X, y = y,training_frame = train,leaderboard_frame = valid)
lb = aml.leaderboard

AutoML progress: |████████████████████████

In [ ]:
lb

In [ ]:
m = aml.leader

In [ ]:
t = m.predict(test)

In [ ]:
pred = t.as_data_frame()['predict']
true = test.as_data_frame()['target']

In [ ]:
print(classification_report(true,pred.values))